<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/ANMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANMI - Tema 3: Aproximación de autovalores

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np

from scipy.linalg import orth

In [ ]:
from anmi.genericas import norma, print_verbose

from anmi.T2 import factorizacion_QR
from anmi.T3 import matriz_krylov, sucesion_krylov, potencia_iterada, metodo_autovals_QR

### Sucesiones de Krylov

In [ ]:
# EJERCICIO 26
A = mat([[1, 1, 1], [0, 2, 2], [3, -1, 0]])
x = mat([[1, 0, 0]]).T

m_krylov = matriz_krylov(A, x)
m_krylov

In [ ]:
sk, a = sucesion_krylov(A, x)
sk

In [ ]:
# EJEMPLO 15
A = mat([[2, -1, 0], [-1, 2, -1], [0, -1, 2]])
x = mat([[-1, 0, 1]]).T

matriz_krylov(A, x)

### Método de la potencia iterada

En el método de la potencia iterada, se aplica la matriz de krylov hasta una potencia determinada, $k$. Entonces, se tiene que 
$$\lim_{k \to \infty} \frac{A^kw}{A^{k-1}w} = |\lambda_1|$$
Es decir, el mayor autovalor.

Además, si tomamos $ B= A^{-1} $, tenemos que
$$\lim_{k \to \infty} \frac{B^kw}{B^{k-1}w} = \frac{1}{|\lambda_n|}$$
Donde $\lambda_n$ es el menor autovalor.

In [ ]:
A.eigenvals()

In [ ]:
x = mat([[-2, 0, 1]]).T

matriz_krylov(A, x, 17)

In [ ]:
x = mat([[-1, 0, 0]]).T

np.array(potencia_iterada(A, x, 30, devolver_ultimo=False)[:, -3:], dtype=float)

In [ ]:
x = mat([[-1, 0, 0]]).T

np.array(potencia_iterada(A, x, 300, devolver_ultimo=True), dtype=float)

In [ ]:
N(2+sqrt(2))  # Vemos que converge al mayor autovalor

In [ ]:
np.array(potencia_iterada(A**-1, x, 300, devolver_ultimo=True), dtype=float)

In [ ]:
1/N(2-sqrt(2))  # Y lo mismo con el menor

In [ ]:
# Si tomamos una matriz ortogonal, el metodo de la potencia no tiene validez 
# porque se requiere que haya autovalores dominantes, y en este caso los 
# autovalores tienen módulo 1.

dict_QR = factorizacion_QR(A)
Q = dict_QR['Q']

Q

In [ ]:
Q * Q.T

In [ ]:
Q.eigenvals()

In [ ]:
N(-1/2 + 3*sqrt(70)/70 + 3*sqrt(14)/28 + sqrt(5)/5 + sqrt(70)*I*sqrt(6*sqrt(14) + 20*sqrt(5) + 73)/140)

In [ ]:
matriz_krylov(Q, x, 5)
N(matriz_krylov(N(Q), x, 30), 4)

In [ ]:
potencia_iterada(N(Q), x, 100, devolver_ultimo=False)[:, -5:]  # No hay una convergencia

### Método QR

El método QR consiste en aplicar los siguientes pasos:

$$A^{(1)} = A$$

De ahí sacamos que 
$$A^{(1)}  = Q^{(1)}R^{(1)}$$

De ahí construimos:
$$A^{(2)}  = R^{(1)}Q^{(1)}$$

Y se cumple que $A^{(1)}$ y $A^{(2)}$ son semejantes, luego tienen los mismos 
autovalores.

Con ello se reitera el proceso, y se cumple que las matrices equivalentes 
construidas convergen a una matriz triangular superior. Los la diagonal de $A^{(k)}$ converge a los autovalores de $A$.

In [ ]:
dict_QR = metodo_autovals_QR(A, n_iters=10)

In [ ]:
N(dict_QR['A'][-2], 3)

In [ ]:
N(dict_QR['A'][-1], 3)

In [ ]:
N(2- sqrt(2), 3), 2, N(2 + sqrt(2), 3), 

In [ ]:
A = mat([[1, 1, 1], [0, 0, 1], [0, 1, 1]])
dict_QR = metodo_autovals_QR(A, n_iters=30, verbose=False)

In [ ]:
N(dict_QR['A'][-15], 3)

In [ ]:
N(dict_QR['A'][-1], 3)

In [ ]:
[N(i, 3) for i in list(A.eigenvals().keys())]